# Jigsaw generation #1

In [1]:
import numpy as np

from tqdm import tqdm

from jigsaw import *
from jigsaw.default import *
from jigsaw.svg import *

In [2]:
def is_unique(horizontal_edges, vertical_edges):
    H, _ = horizontal_edges.shape
    _, W = vertical_edges.shape
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    return has_unique_solution(H, W, pieces, opposite, flip, constraints="border")


def search(sampler, *validators):

    # Loop until a grid with a unique solution is found
    with tqdm() as progress:
        while True:

            # Sample grid, according to rules
            horizontal_edges, vertical_edges = sampler()
            
            # Ask validators for compliance
            try:
                for validator in validators:
                    if not validator(horizontal_edges, vertical_edges):
                        break
                else:
                    return horizontal_edges, vertical_edges
            finally:
                progress.update(1)

## Simple 3x3

In [3]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    CENTERED_SMALL_MALE,
    CENTERED_MEDIUM_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_MEDIUM_MALE', 'CENTERED_MEDIUM_FEMALE',
       'CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE'], dtype='<U22')

In [4]:
# Generate grid
H, W = 3, 3
generator = np.random.default_rng(42)
sampler = lambda: sample_random_grid(H, W, edge_types, generator=generator)
horizontal_edges, vertical_edges = search(sampler, is_unique)

3it [00:03,  1.03s/it]


In [5]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [6]:
# Keep it for later
pieces_3x3 = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## Skewed 3x3

In [7]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    LEFT_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['RIGHT_SMALL_MALE', 'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE',
       'LEFT_SMALL_FEMALE'], dtype='<U22')

In [8]:
# Generate grid
H, W = 3, 3
generator = np.random.default_rng(42)
sampler = lambda: sample_random_grid(H, W, edge_types, generator=generator)
horizontal_edges, vertical_edges = search(sampler, is_unique)

2it [00:00, 153.84it/s]


In [9]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [10]:
# Keep it for later
pieces_3x3_skewed = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## Simple 4x4

In [11]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    CENTERED_SMALL_MALE,
    CENTERED_MEDIUM_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_MEDIUM_MALE', 'CENTERED_MEDIUM_FEMALE',
       'CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE'], dtype='<U22')

In [12]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(42)
sampler = lambda: sample_random_grid(H, W, edge_types, generator=generator)
horizontal_edges, vertical_edges = search(sampler, is_unique)

144it [00:00, 180.38it/s]


In [13]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [14]:
# Keep it for later
pieces_4x4 = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 4x4 with 8 corners (no constraint)

In [15]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    CENTERED_SMALL_MALE,
    LEFT_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE', 'RIGHT_SMALL_MALE',
       'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE', 'LEFT_SMALL_FEMALE'],
      dtype='<U22')

In [16]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(42)
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    horizontal_edges[1:-1, 2] = FLAT
    vertical_edges[2, 1:-1] = FLAT
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler, is_unique)

6it [00:00, 65.90it/s]


In [17]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [18]:
# Keep it for later
pieces_4x4_cross = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 4x4 with 8 corners (with constraints)

In [19]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(12)
def sampler():
    
    # Start from a regular grid, composed only of small left/right pins
    edge_types = np.array([
        LEFT_SMALL_MALE,
        LEFT_SMALL_FEMALE,
        RIGHT_SMALL_MALE,
        RIGHT_SMALL_FEMALE,
        DOUBLE_SMALL_MALE,
        DOUBLE_SMALL_FEMALE,
    ])
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    
    # Add cross in the middle
    horizontal_edges[1:-1, 2] = FLAT
    vertical_edges[2, 1:-1] = FLAT
    
    # Make sure "center" corners are connected only using centered pins
    edge_types = np.array([CENTERED_SMALL_MALE, CENTERED_SMALL_FEMALE])
    horizontal_edges[1, 1] = generator.choice(edge_types)
    horizontal_edges[2, 1] = generator.choice(edge_types)
    horizontal_edges[1, 3] = generator.choice(edge_types)
    horizontal_edges[2, 3] = generator.choice(edge_types)
    vertical_edges[1, 1] = generator.choice(edge_types)
    vertical_edges[1, 2] = generator.choice(edge_types)
    vertical_edges[3, 1] = generator.choice(edge_types)
    vertical_edges[3, 2] = generator.choice(edge_types)

    return horizontal_edges, vertical_edges

horizontal_edges, vertical_edges = search(sampler, is_unique)

116it [00:00, 170.97it/s]


In [20]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [21]:
# Using all available pieces
pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

# For each corner
for k in range(4):
    
    # Rotate grid, such that corner is placed top-left
    h, v = rotate_grid(horizontal_edges, vertical_edges, opposite, k)
    display_grid(h, v)
    
    # The 2x2 area top-left is used as a constraint
    hc = h[:2, :3].astype(int).copy()
    vc = v[:3, :2].astype(int).copy()
    hc[0, 2] = -1
    hc[1, 1] = -1
    vc[2, 0] = -1
    vc[1, 1] = -1
    constraints = hc, vc
    display_grid(hc, vc)
    
    # Count solutions; we would like only one per corner
    count = 0
    for g in iterate_solutions(2, 2, pieces, opposite, flip, constraints=constraints):
        count += 1
        display_grid(*g)
    print("->", count)

-> 8


-> 3


-> 3


-> 3


In [22]:
# Wrap that as a function
def has_unique_corner(horizontal_edges, vertical_edges, k):
    
    # Using all available pieces
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

    # Rotate grid, such that corner is placed top-left
    h, v = rotate_grid(horizontal_edges, vertical_edges, opposite, k)

    # The 2x2 area top-left is used as a constraint
    hc = h[:2, :3].astype(int).copy()
    vc = v[:3, :2].astype(int).copy()
    hc[0, 2] = -1
    hc[1, 1] = -1
    vc[2, 0] = -1
    vc[1, 1] = -1
    constraints = hc, vc

    # Count solutions; we would like only one per corner
    has = False
    for g in iterate_unique_solutions(2, 2, pieces, opposite, flip, constraints=constraints):
        if has:
            return False
        has = True
    return has

def has_unique_corners(horizontal_edges, vertical_edges):
    for k in range(4):
        if not has_unique_corner(horizontal_edges, vertical_edges, k):
            return False
    return True

In [23]:
has_unique_corner(horizontal_edges, vertical_edges, 0)

False

In [24]:
horizontal_edges, vertical_edges = search(sampler, is_unique, has_unique_corners)

2440it [00:12, 195.26it/s]


In [25]:
display_grid(horizontal_edges, vertical_edges)

In [26]:
# Keep it for later
pieces_4x4_cross_constrained = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 5x5 with square center

In [27]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    CENTERED_SMALL_MALE,
    LEFT_SMALL_MALE,
    DOUBLE_SMALL_MALE,
    #TWISTED_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE', 'RIGHT_SMALL_MALE',
       'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE', 'LEFT_SMALL_FEMALE',
       'DOUBLE_SMALL_MALE', 'DOUBLE_SMALL_FEMALE'], dtype='<U22')

In [28]:
# Generate grid
H, W = 5, 5
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types)
    horizontal_edges[2, 2:-2] = FLAT
    vertical_edges[2:-2, 2] = FLAT
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler)

1it [00:00, 987.82it/s]


In [29]:
display_grid(horizontal_edges, vertical_edges)

In [30]:
# TODO

## Twisted 4x4

In [31]:
# Choose only centered edge types, two sizes
edge_types = np.array([
    LEFT_SMALL_MALE,
    TWISTED_SMALL_MALE,
    DOUBLE_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['RIGHT_SMALL_MALE', 'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE',
       'LEFT_SMALL_FEMALE', 'DOUBLE_SMALL_MALE', 'DOUBLE_SMALL_FEMALE',
       'TWISTED_SMALL_MALE', 'TWISTED_SMALL_FEMALE'], dtype='<U22')

In [32]:
# Generate grid
H, W = 4, 4
sampler = lambda: sample_random_grid(H, W, edge_types, generator=generator)
horizontal_edges, vertical_edges = search(sampler)

1it [00:00, ?it/s]


In [33]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [34]:
# TODO improve it

In [35]:
# Keep it for later
pieces_4x4_twisted = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 3x3 + 4x4 = 5x5

In [36]:
# TODO

## Pack as a single grid

In [37]:
# Pack jigsaw puzzles as a single grid
pieces = np.zeros((7, 8, 4), dtype=np.uint8)
pieces[:4, :4] = pieces_4x4
pieces[:4, 4:8] = pieces_4x4_twisted
pieces[4:7, :3] = pieces_3x3
horizontal_edges, vertical_edges = pieces_to_grid(pieces, opposite)
display_grid(horizontal_edges, vertical_edges)

In [38]:
# Export as SVG
with open("attempt_01.svg", "w") as file:
    file.write(grid_to_svg(horizontal_edges, vertical_edges))